In [26]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(queue='analysis',memory='60GB',project='gfdl_f',cores=8, walltime='0:60:00' )

/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [27]:
cluster

distributed.utils - ERROR - 'ogb'
Traceback (most recent call last):
  File "/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 714, in log_errors
    yield
  File "/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/dask_jobqueue/deploy/cluster_manager.py", line 166, in _scale
    n = self._get_nb_workers_from_memory(memory)
  File "/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/dask_jobqueue/deploy/cluster_manager.py", line 377, in _get_nb_workers_from_memory
    parse_bytes(memory) / parse_bytes(self.jobqueue_worker_spec["memory"])
  File "/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/distributed/utils.py", line 1173, in parse_bytes
    multiplier = byte_sizes[suffix.lower()]
KeyError: 'ogb'
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x2b1395e0eb70>>, <Future finished exception=KeyError('ogb')>)
Traceback (most recent

In [ ]:
# cluster.close()

In [30]:
from dask.distributed import Client
client = Client(cluster)

In [31]:
client

Client Scheduler: tcp://140.208.147.176:33121 Dashboard: http://140.208.147.176:43617/status,Cluster Workers: 1 Cores: 8 Memory: 60.00 GB


In [33]:
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import xarray as xr
import numpy

In [34]:
#300yr OM4files, date 198801 - 200712

#chunk along dim = 'time'
OM4p25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p25_tos_300yr.zarr')
OM4p25z = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p25z_tos_300yr.zarr')
OM4p5nep = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p5nep_tos_300yr.zarr')
OM4p5mle =xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p5mle_tos_300yr.zarr')
OM4p5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/OM4p05_tos_300yr.zarr')

#WOA files
W25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA25.zarr')
W5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA05.zarr')
# 105GB < T_fileSize <110GB

#Grid files
G25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid25.zarr')
G5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid05.zarr')
M25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask25.zarr')
M5 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask05.zarr')

In [35]:
xq25 = G25['x'][::2,::2]
yq25 = G25['y'][::2,::2]
a25 = G25['area']; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25['mask']; a25 = a25*m25
xq5 = G5['x'][::2,::2]
yq5 = G5['y'][::2,::2]
a5 = G5['area']; a5 = a5[::2,::2]+a5[1::2,1::2]+a5[1::2,::2]+a5[1::2,::2]
m5 = M5['mask']; a5 = a5*m5

/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


### Time Average

In [36]:
##xarray lazy computation

In [37]:
OM4p5_av = OM4p5.variables['tos'].chunk({'time':12}).mean(dim='time', skipna=True).load()
OM4p5_av = OM4p5.variables['tos'].chunk({'time':12}).mean(dim='time', skipna=True).load()
OM4p25_av = OM4p25.variables['tos'].chunk({'time':12}).mean(dim='time', skipna=True).load()
OM4p5nep_av = OM4p5nep.variables['tos'].chunk({'time':12}).mean(dim='time', skipna=True).load()
OM4p25z_av = OM4p25z.variables['tos'].chunk({'time':12}).mean(dim='time', skipna=True).load()

In [38]:
W25_av = W25.variables['ptemp'].mean(axis=(0,1), skipna=True).load()
W5_av = W5.variables['ptemp'].mean(axis=(0,1), skipna=True).load()

In [39]:
# %%time
# OM4p25_av = OM4p25['tos'].mean(dim='time').load()
# OM4p5nep_av = OM4p5nep['tos'].mean(dim='time').load()
# OM4p25z_av = OM4p25z['tos'].mean(dim='time').load()
# OM4p5_av = OM4p5['tos'].mean(dim='time').load()
# OM4p5_av = OM4p5['tos'].mean(dim='time').load()

/nbhome/Nkeh.Boh/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")


In [40]:
def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')

In [ ]:
fig = plt.figure(figsize=(12, 8))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = OM4p5nep_av - W5_av
ax = fig.add_subplot(2,2,2,projection=cartopy.crs.Robinson(central_longitude=-155))
im = ax.pcolormesh(xq5, yq5, q,
              transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
stats(ax, a5, q, 'OMp5nep')
ax.coastlines()
axes.append(ax)
plt.title('b) OM4p5n')

# q = OM4p25z_av - W25_av
# ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq25, yq25, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a25, q, 'OMp25z')
# ax.coastlines()
# axes.append(ax)
# plt.title('c) OM4p25z')

# q = OM4p5mle.variables['tos'][0,0]-W5.variables['ptemp'][0,0]
# ax = fig.add_subplot(2,2,3,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq5, yq5, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm)
# stats(ax, a5, q, 'OM4p5mle')
# ax.coastlines()
# axes.append(ax)
# plt.title('c) OM4p25e')

# q = OM4p5_av - W5_av
# ax = fig.add_subplot(2,2,4,projection=cartopy.crs.Robinson(central_longitude=-155))
# im = ax.pcolormesh(xq5, yq5, q,
#               transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
# stats(ax, a5, q, 'OMp5')
# ax.coastlines()
# axes.append(ax)
# plt.title('d) OM4p5')

# ax = plt.gcf().add_axes((.25,.5,.5,.03))
# cb = matplotlib.colorbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,
#                                       orientation='horizontal', extend='both')
# cb.ax.set_title('SST bias ($^\circ$C)')

# plt.savefig('sst-bias.png')